In [1]:
import wandb
import numpy as np
# Set your API key
wandb.login()

# Set the source and destination projects
src_entity = "cl-probing"
src_project = "contrastive_fns"
dst_entity = "cl-probing"
dst_project = "logsumexp_penalty"

# Initialize the wandb API
api = wandb.Api()

# Get the runs from the source project
exp_names = ["ant_l_symmetric_infonce_ef_l2"]

filters = {
    "config.batch_size": 256,  # example filter, modify as needed
    "config.exp_name": {"$in": exp_names},
    "state": "finished"
}
runs = api.runs(path=f"{src_entity}/{src_project}", filters=filters)
name = "ant_l_symmetric_infonce_ef_l2_p_0.1"

print(len(runs))

/home/mbortkie/miniconda3/envs/c_r_l/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/mbortkie/miniconda3/envs/c_r_l/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
wandb: Currently logged in as: michalbortkiewicz (cl-probing). Use `wandb login --relogin` to force relogin


20


In [2]:
# Iterate through the runs and copy them to the destination project
for run in runs: 
    history = run.history()
    files = run.files()


    # Create a new run in the destination project

    # Log the history to the new run
    new_run = wandb.init(
        project=dst_project,
        entity=dst_entity,
        config=run.config,
        name=name,
        resume="allow"
    )
    for index, row in history.iterrows():
        step_size = history['_step'].values[1]
        new_run.log(row.to_dict(), step= int(index * step_size)//2)

    # Upload the files to the new run
    for file in files:
        file.download(replace=True)
        new_run.save(file.name, policy="now")

    # Finish the new run
    new_run.finish()


eval/episode_dist,▂▃██▆▆▅▅▇▄▅▄▂▂▂▂▃▂▂▂▃▂▃▁▂▃▁▂▂▃▃▂▂▂▂▃▄▂▃▂
eval/episode_reward_survive,▂▂▇█▇█▄▅▇▄▅▄▂▃▂▂▂▃▄▃▃▄▃▁▂▃▂▃▂▃▃▁▂▃▃▃▅▃▃▂
eval/episode_success,▁▁▂▃▄▄▂▄▃▄▃▃▃▅▃▅▂▆▆▄▄█▄▄▃▄▄▃▄▃▄▃▄▃▄▃▃▅▃▂
eval/episode_success_any,▁▁▂▄▄▆▃▅▄▅▄▄▄▆▄▆▃▆█▅▅█▄▆▅▆▅▅▆▄▅▃▅▇▅▅▅▇▄▃
eval/episode_success_easy,▁▁▂▅▅▅▃▅▄▄▃▄▄▆▄▆▃▆▇▅▄█▄▄▄▅▅▄▅▄▄▃▄▅▄▄▄▅▃▃
eval/episode_success_hard,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
training/actor_loss,▁▇███▇▇█████████████████████████████████
training/alpha,▁█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
training/alpha_loss,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training/binary_accuracy,▁███████████████████████████████████████


eval/episode_dist,▁█▄▅▆▅▅▆▄▅▂▃▅▄▄▃▃▃▃▂▃▃▄▃▃▅▄▃▄▂▄▄▄▃▄▃▂▃▃▂
eval/episode_reward_survive,▁█▅▄▇▅▆█▇▆▄▆▇▆▆▇▆▅▅▅▅▅▇▆▅▇▇▆▆▅▇▆▇▅▆▅▄▅▅▅
eval/episode_success,▁▂▂▁▃▃▃▅▅▅▅▇▆▇▅██▇▄▄▄▅▅▆▅▆▇▅▅▆▆▆▅▄▅▅▄▅▆▆
eval/episode_success_any,▁▂▃▂▄▄▃▅▆▄▆▆▅▆▆█▇▆▅▆▆▆▆█▆▆▆▆▅█▇▆▅▄▅▅▄▅▇▇
eval/episode_success_easy,▁▂▄▂▅▄▄▆▆▄▅▇▆▆▅██▇▄▅▅▆▆▇▆▆▆▆▅▇▆▇▅▄▅▅▄▅▇▇
eval/episode_success_hard,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
training/actor_loss,▁███▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
training/alpha,▁█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
training/alpha_loss,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training/binary_accuracy,▁███████████████████████████████████████


eval/episode_dist,▁█▄▆▆▄▄▆▆▅▆▃▂▃▄▄▃▄▄▃▃▄▄▃▂▃▃▃▄▂▃▅▅▄▄▅▄▅▄▃
eval/episode_reward_survive,▁█▄▆▇▇▇█▇▇▇▅▃▄▇▆▆▆▆▇▆▇▆▆▅▆▆▆▅▆▅▇▇▆▇▇▆▇▇▆
eval/episode_success,▁▁▁▂▁▃▃▂▂▂▃▃▃▅▄▃▅▄▄▅▅▆▄▅▄▅▄▄▂█▄▄▃▄▅▄▅▅▄▄
eval/episode_success_any,▁▂▂▃▂▄▆▄▃▄▃▅▃▄▅▄▅▆▅▅▆▆▄▅▅▆▅▅▄█▅▅▄▅▆▅▅▄▅▆
eval/episode_success_easy,▁▂▂▃▂▄▅▃▃▃▃▄▃▄▄▄▅▄▄▅▆▆▄▅▅▆▅▅▃█▄▅▄▄▅▅▄▄▅▅
eval/episode_success_hard,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
training/actor_loss,▁▇█████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
training/alpha,▁█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
training/alpha_loss,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training/binary_accuracy,▁███████████████████████████████████████


eval/episode_dist,▁█▇█▆▆▅▆▆▅▅▆▅▅▄▂▄▅▅▄▄▅▅▅▆▅▄▃▄▄▃▄▄▅▅▄▃▄▄▃
eval/episode_reward_survive,▁▇▇█▇▇▆▆▇▆▇█▇▇▆▅▇▅▇▆▅▇▆▇▇▇▅▆▆▅▅▆▆▇▇▅▅▆▅▅
eval/episode_success,▁▁▁▃▂▂▃▃▅▆▄▅▅▇▆▅█▄▅▆▇▇▆█▅█▄▇▄▅▆▄▇▅▇▆▅▃▅▃
eval/episode_success_any,▁▂▂▄▄▃▄▄▆▆▅▆▇▇▇▇█▅▆▆▆▇▅█▅▇▅▆▅▄▆▅▅▆▆▆▅▅▅▄
eval/episode_success_easy,▁▂▃▄▄▃▄▄▆▇▅▆▆▇▆▇█▄▆▆▆▇▆█▅█▄▇▅▄▆▅▆▅▆▅▅▃▅▄
eval/episode_success_hard,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
training/actor_loss,▁▇███▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
training/alpha,▁█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
training/alpha_loss,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training/binary_accuracy,▁███████████████████████████████████████


eval/episode_dist,▄█▅▇▆▅▆▄▆▃▄▄▂▃▄▃▂▄▃▃▃▄▃▃▃▃▂▁▃▂▂▃▃▃▃▄▃▂▂▂
eval/episode_reward_survive,▁█▄▇▅▆█▆▇▄▅▅▃▅▆▅▆█▆▅▅▇▅▆▅▅▄▄▅▄▄▅▅▅▆▇▆▄▆▃
eval/episode_success,▁▁▁▁▃▃▄▃▅▄▆▅▅▅▆▅▆▇▆▄▆▄▆▆▆▅▄█▅▅▆▆▅▅▇▆▇▇▇▅
eval/episode_success_any,▁▁▂▂▄▄▅▅▅▄▆▅▅▅▆▅▇▇▅▅▆▄▆▇▆▆▅▇▅▅▅▆▅▅▇▆▇██▆
eval/episode_success_easy,▁▁▂▂▄▃▄▄▅▄▆▄▄▅▅▅▆▇▅▅▆▄▆▆▆▅▅█▅▄▆▅▅▅▆▅▆▇▇▆
eval/episode_success_hard,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
training/actor_loss,▁███▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
training/alpha,▁█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
training/alpha_loss,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training/binary_accuracy,▁███████████████████████████████████████


...

eval/episode_dist,▇█▄▂▂▁▁▁▁▁▁▂▁▂▂▂▂▂▁▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
eval/episode_reward_survive,▂▄▄▁▆▅▅▆▇▇▇▇▅█▆█▅▅▅▇▆▇▆▇▇▆▇▇▆▇▆▇▇▆▆▅▅▅▅▅
eval/episode_success,▁▁▁▂▄▄▅▆█▇▇▆▅▆▆▇▄▄▅▆▅▆▅▅▆▅▆▆▅▅▆▆▅▅▆▅▅▄▅▅
eval/episode_success_any,▁▁▂▃▆▆▆▆▇▇▇█▇▇▇█▆▆▇▇▇█▇▇▇▇▇█▇▇▇▇▇▇▇▆▇▇▇▇
eval/episode_success_easy,▁▁▂▃▆▇▆▇████▆▇▇█▆▆▆▇▆▇▇▇▇▆▇▇▇▇▇▇▇▆▆▆▆▅▆▆
eval/episode_success_hard,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
training/actor_loss,▁███▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
training/alpha,▁█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
training/alpha_loss,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training/binary_accuracy,▁███████████████████████████████████████


eval/episode_dist,▃█▄▂▂▁▁▁▁▁▁▁▁▁▂▁▁▂▁▂▁▁▂▂▁▂▁▂▁▂▂▂▁▁▁▂▁▂▂▂
eval/episode_reward_survive,▁▇▅▅▆▅▆▅▇▆▆▆▆▆██▇█▇▇▇▆█▇▇█▇▇▆▇▇▆▇▆▇▇▇▇▇▇
eval/episode_success,▁▁▁▂▃▃▅▅▆▆▇▆▆▆▇█▇█▇▇█▇█▆███▇▆▇▇▇▇▆▇▇▇▇▇▆
eval/episode_success_any,▁▁▂▄▆▅▇▆▇▇▇▇▇▇▇█▇████▇▇▇▇█▇█▆▇█▇▇▇█▇▇▇█▆
eval/episode_success_easy,▁▁▂▄▆▅▆▆▇▇▇▇▆▆▇█▇█▇▇█▇█▆▇█▇▇▆▇▇▇▇▇▇▇▇▇▇▆
eval/episode_success_hard,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
training/actor_loss,▁██▇▇▆▆▆▆▆▆▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
training/alpha,▁█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
training/alpha_loss,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training/binary_accuracy,▁███████████████████████████████████████


eval/episode_dist,▄█▂▃▂▂▂▂▂▁▁▂▁▁▁▁▂▃▁▂▂▁▁▂▂▃▂▂▂▂▂▂▂▂▂▃▂▂▂▃
eval/episode_reward_survive,▁▄▁▅▆▆▇▇▆▇▆▇▇▇▆▇▇█▆▇▇▇▇▇▇█▇▇▇▇▇▇█▇▇▇▇██▇
eval/episode_success,▁▁▁▂▄▄▅▅▅▇▆▇▇█▆▆▇▆▆▆▇█▇▆▇▆▆▇▆▇▇▆█▇▇▆▇▇▇▆
eval/episode_success_any,▁▂▂▄▆▆▆▇▇█▇▇██▆▇▇▇▇▇▇█▇▇▇█▇▇▇▇▇▇█▇█▇▇▇▇▇
eval/episode_success_easy,▁▁▂▄▆▆▆▇▆█▇▇██▆▇▇▇▆▇▇█▇▆▇▇▇▇▇▇▇▇█▇▇▆▆▇▇▆
eval/episode_success_hard,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
training/actor_loss,▁▇██▇▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
training/alpha,▁█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
training/alpha_loss,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training/binary_accuracy,▁███████████████████████████████████████


eval/episode_dist,▄█▆▄▃▂▂▂▁▂▁▁▂▁▃▁▂▂▁▂▂▁▂▁▁▂▁▂▂▁▁▁▁▁▂▁▁▂▁▂
eval/episode_reward_survive,▁▅▅▅▆▆▇▆▇▇▆▆▇▇█▇▇▇▇█▆▆▇▆▆█▇▇▇▆▇▇▆▇█▇▆▇▇▆
eval/episode_success,▁▁▁▂▂▃▄▃▅▅▅▆▆▇▆▅▆▆▇▇▆▇▇▆▆▇▇▇▇▆▇█▆▇█▆▇▇█▆
eval/episode_success_any,▁▁▂▃▅▆▇▆▇▇▇▇▇▇█▇▇▇█▇▇▇▇▇▇█▇▇▇▇▇▇▇▇█▇▇▇▇▇
eval/episode_success_easy,▁▁▂▃▄▅▆▆▇▇▆▇▇█▇▇▇▇▇█▆▇▇▆▆▇▇▇▇▇▇█▇██▇▇▇█▇
eval/episode_success_hard,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
training/actor_loss,▁███▇▇▆▆▆▆▆▆▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
training/alpha,▁█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
training/alpha_loss,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training/binary_accuracy,▁███████████████████████████████████████


eval/episode_dist,█▄▆▆▄▂▃▃▃▃▂▂▂▂▃▂▃▃▁▃▂▂▄▂▂▃▂▂▂▂▁▃▄▃▃▃▄▃▄▂
eval/episode_reward_survive,▂▁▄▆▇▆█▆▇█▆▇█▇▆█▇█▇▇▇▇█▇▇▇▇▇▆▇▆▇▇▇█▇█▇██
eval/episode_success,▁▁▁▂▃▄▅▄▆▇▆▇██▅█▆██▇▇▇▆▇█▇▇▇▇█▇▇▆▇█▆▇▇▇█
eval/episode_success_any,▁▁▂▄▆▆█▆▇▇▆▇█▇▆█▇█▇▇▇▇▇▇██▇▇▇▇▆▇▇▇▇▇▇▇▇▇
eval/episode_success_easy,▁▁▂▄▆▆▇▆▇▇▆▇██▆█▇██▇▇▇▇▇█▇▇▇▇▇▆▇▆▇▇▆▇▇▇█
eval/episode_success_hard,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
training/actor_loss,▁███▇▆▆▆▆▆▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
training/alpha,▁█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
training/alpha_loss,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training/binary_accuracy,▁███████████████████████████████████████
